In [ ]:
import meep as mp
import numpy as np

# imaginary part구하는 함수
def D_conductivity(real, imag, freq):
    return((2*np.pi*freq)*imag)/real

##############################################################################
# 1. 시뮬레이션 기본 파라미터 설정
##############################################################################

resolution = 50     # 1um 당 50픽셀 (약 20nm/픽셀)
dpml = 0.2          # PML 두께(um)

sx = 1.0            # 시뮬레이션 영역 가로 크기(um) - 나노 LED 폭(1,000 nm)이라 가정
sy = 2.5            # 시뮬레이션 영역 세로 크기(um)
cell_size = mp.Vector3(sx, sy, 0)  # 2D 시뮬레이션 (z=0)

pml_layers = [mp.PML(dpml, direction=mp.Y)]

# 소스 파장: 450 nm => 0.45 um
wavelength = 0.45
fcen = 1 / wavelength   # 중심 주파수
df = 0.0                # 단색광 사용

##############################################################################
# 2. 재료(복소수 유전율) 정의
##############################################################################
SiO2  = mp.Medium(epsilon=2.13)
GaN   = mp.Medium(epsilon=6.18)
Al2O3 = mp.Medium(epsilon=3.17)

# Ag는 complex기 때문에 따로 정의 Ag = -7.01 + i0.21
real = 49.096
imag = -2.9442
D_con = D_conductivity(real, imag, fcen)
Ag    = mp.Medium(epsilon=real, D_conductivity=D_con)

##############################################################################
# 3. 구조(Geometry) 정의
##############################################################################
# 각 층 두께(um)
t_qw    = 0.01   # QW 10 nm 예시
t_sio2  = 0.07   # 70 nm
t_al2o3 = 0.10   # 100 nm
t_ag    = 0.05   # Ag 층 50 nm 예시 (질문 속 Ag 두께는 정확히 명시 안 됨)

# 레이어 위치(y좌표)
y_qw_center = 0.0
y_qw_min = y_qw_center - t_qw/2
y_qw_max = y_qw_center + t_qw/2

y_sio2_min = y_qw_max
y_sio2_max = y_sio2_min + t_sio2

y_al2o3_min = y_sio2_max
y_al2o3_max = y_al2o3_min + t_al2o3

y_ag_max = y_qw_min
y_ag_min = y_ag_max - t_ag

geometry = [
    # QW (InGaN/GaN 혼합영역이라 가정, 단순화해서 GaN으로)
    mp.Block(
        size=mp.Vector3(sx, t_qw, mp.inf),
        center=mp.Vector3(0, y_qw_center),
        material=GaN
    ),
    # SiO2 Layer
    mp.Block(
        size=mp.Vector3(sx, t_sio2, mp.inf),
        center=mp.Vector3(0, 0.5*(y_sio2_min + y_sio2_max)),
        material=SiO2
    ),
    # Al2O3 Layer
    mp.Block(
        size=mp.Vector3(sx, t_al2o3, mp.inf),
        center=mp.Vector3(0, 0.5*(y_al2o3_min + y_al2o3_max)),
        material=Al2O3
    ),
    # Ag 반사층
    mp.Block(
        size=mp.Vector3(sx, t_ag, mp.inf),
        center=mp.Vector3(0, 0.5*(y_ag_min + y_ag_max)),
        material=Ag
    )
]

##############################################################################
# 4. 소스(Dipole) 정의
##############################################################################
# QW 영역( y=0 인근 )에서 dipole 소스 1개 배치 (2D TM 모드, Ez 사용)
sources = [
    mp.Source(
        src=mp.GaussianSource(frequency=fcen, fwidth=0.1*fcen),
        component=mp.Ez,
        center=mp.Vector3(0, 0)  # QW 중앙에 배치
    )
]

##############################################################################
# 5. Simulation 객체 생성
##############################################################################
sim = mp.Simulation(
    cell_size=cell_size,
    boundary_layers=pml_layers,
    geometry=geometry,
    sources=sources,
    resolution=resolution
)

##############################################################################
# 6. Flux 모니터 설정 & 추가
##############################################################################
# (1) 소스 주변(Input flux) 모니터 (Dipole 주위)
monitor_in_region = mp.FluxRegion(
    center=mp.Vector3(0, 0),        # 모니터 중심 (QW 중앙 근방)
    size=mp.Vector3(sx, 0, 0)       # x방향 폭: sx, y방향: 0 (1D 선 형태)
)

flux_in = sim.add_flux(
    fcen,   # 중심 주파수
    df,     # 주파수 대역폭
    1,      # 주파수 샘플 개수
    monitor_in_region
)

# (2) Planewave Monitor: QW 상부 1λ(=0.45um) 위쪽
planewave_y = y_al2o3_max + 0.45  # QW 상부에서 0.45 um 정도 떨어진 위치
monitor_out_region = mp.FluxRegion(
    center=mp.Vector3(0, planewave_y),
    size=mp.Vector3(sx, 0, 0)
)

flux_out = sim.add_flux(
    fcen,
    df,
    1,
    monitor_out_region
)

##############################################################################
# 7. 시뮬레이션 실행
##############################################################################
simulation_time = 200  # 충분히 길게
sim.run(until=simulation_time)

##############################################################################
# 8. 모니터 결과 출력
##############################################################################
# get_fluxes() → 각 모니터에서 주파수별 플럭스(여기서는 1개 주파수만)
in_flux_data = mp.get_fluxes(flux_in)
out_flux_data = mp.get_fluxes(flux_out)

p_in  = in_flux_data[0]
p_out = out_flux_data[0]

print(f"Input flux  (P_in): {p_in}")
print(f"Planewave flux (P_out): {p_out}")

##############################################################################
# 9. Far-field/Angle 계산 (추가 예시)
##############################################################################
# ※ 아래는 실제 실행 예시가 아니라, 파-장(Planewave) 모니터로부터
#   farfield 변환을 수행하는 전형적인 패턴입니다.
#   필요에 따라 각도별 E/H field나 방출강도를 산출할 수 있습니다.

# 예) farfield_points = [mp.Vector3( r*np.sin(theta), planewave_y + r*np.cos(theta) )
#                       for theta in np.linspace(-np.pi/2, np.pi/2, 200)]
# ff_data = sim.get_farfield(flux_out, farfield_points)

# ±30° 범위의 방출을 구하려면, theta=-30°~+30° 범위에서 ff_data를 적분/합산하는 방식.


